In [1]:
import encoding_tools
from pathlib import Path
import numpy as np
import pickle
import pandas as pd

# ML imports
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance
from scipy import optimize, linalg
import scipy
from sklearn.model_selection import KFold # import KFold

# custom imports
import encoding_tools as encoding
import chimera_tools as chimera
import GP_tools as GP

C:\Users\kikir\anaconda3\envs\GP_Model\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
Processed_Folder = Path(r"Phosphotase_Encode.ipynb").parent.absolute() / Path("Processed Data")

dicts = ['EFI_ID_List', 'metabolite_dict', 'Protein_seq_dict']

with open(Processed_Folder / Path('EFI_ID_List.p'), 'rb') as EFI_ID:
    EFI_ID_List = pickle.load(EFI_ID)

with open(Processed_Folder / Path('metabolite_dict.p'), 'rb') as metabolite:
    metabolite_dict = pickle.load(metabolite)

with open(Processed_Folder / Path('Protein_seq_dict.p'), 'rb') as Protein_seq:
    Protein_seq_dict = pickle.load(Protein_seq)

activations = pd.read_csv(Processed_Folder / Path('activations.csv'), index_col=0)

In [3]:
# Need to pad protein sequences to the max length of the longest one
max_len = len(max(Protein_seq_dict.values(), key=len))
fillchar = '-' # This is whats used in the GP-UCB paper
Padded_dict = {}
OH_dict = {}
for ID in EFI_ID_List:
    Padded_dict[ID] = Protein_seq_dict[ID].upper().ljust(max_len, fillchar)
    OH_dict[ID] = encoding_tools.one_hot_seq(seq_input=Padded_dict[ID])

In [10]:
# Preparing input training data X to feed into ML Model
input_len = len(OH_dict[EFI_ID_List[0]])*21
num_inputs = len(OH_dict.keys())

X = np.zeros((num_inputs,input_len))
for i in range(0,len(EFI_ID_List)):
    ID = EFI_ID_List[i]
    X_seq = OH_dict[ID]
    X_seq = np.reshape(X_seq,(1,X_seq.shape[0]*21))
    X[i,:] = X_seq

# Preapre output training data y to feed into ML Model
y = activations.values[0,:]

In [5]:
ID = np.random.randint(low=0,high=218)
len_comp = len(Padded_dict[EFI_ID_List[ID]])
len(Padded_dict[EFI_ID_List[0]]) == len(Padded_dict[EFI_ID_List[ID]])
print(len_comp)
print(ID) 

604
206


In [6]:
def ML_train(X, y):
    # test the optimization of the hyp-prams
    initial_guess = [0.9,0.9]

    # take the log of the initial guess for optimiziation 
    initial_guess_log = np.log(initial_guess)

    # optimize to fit model
    result = scipy.optimize.minimize(GP.neg_log_marg_likelihood, initial_guess_log, args=(X,y), method='L-BFGS-B')
    
    print('Full GP regression model')
    print('Hyperparameters: ' + str(np.exp(result.x[0])) + ' ' + str(np.exp(result.x[1])))

    # next set of hyper prams 
    final_prams = [np.exp(result.x[0]), np.exp(result.x[1])]
    
    return final_prams
    

In [13]:
def cross_validation(X, log_data, property_):
    path_outputs = 'outputs/'

    kf = KFold(n_splits=20) # Define the split
    kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

    mu_s = []
    var_s = []
    y_s = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]

        log_data_train, log_data_test = log_data[train_index], log_data[test_index]

        y_train = (log_data_train - np.mean(log_data_train))/np.std(log_data_train)
        y_test = (log_data_test - np.mean(log_data_train))/np.std(log_data_train)

        initial_guess = [0.1,10]

        # take the log of the initial guess for optimiziation 
        initial_guess_log = np.log(initial_guess)

        # optimize to fit model
        result = scipy.optimize.minimize(GP.neg_log_marg_likelihood, initial_guess_log, args=(X_train,y_train), method='L-BFGS-B')#,

        # next set of hyper prams 
        prams_me = [np.exp(result.x[0])**2, np.exp(result.x[1])]

        # next used trained GP model to predict on test data
        mu, var = GP.predict_GP(X_train, y_train, X_test, prams_me)
        
        # un normalize
        y_test_real = np.exp(y_test*np.std(log_data_train)  + np.mean(log_data_train))
        mu_real = np.exp(mu*np.std(log_data_train)  + np.mean(log_data_train))
        
        mu_s.append(mu)
        var_s.append(var)
        y_s.append(y_test)

    # reformat all
    y_s_all = [j for i in y_s for j in i]
    mu_s_all = [j for i in mu_s for j in i]

    # plot results
    plt.figure('My GP test set evaluation', figsize=(1.5, 1.5))
    plt.plot(y_s_all, mu_s_all, 'o', ms=3, color='k')


    # calculate correlation 
    measured = y_s_all
    predicted = mu_s_all

    par = np.polyfit(measured, predicted, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]

    # calc correlation 
    variance = np.var(predicted)
    residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(measured, predicted)])
    Rsqr = np.round(1-residuals/variance, decimals=2)
    
    print('20-fold corss validation of GP regression model')
    print('R = %0.2f'% np.sqrt(Rsqr))

    max_x = np.max(y_s_all)
    min_x = np.min(y_s_all)
    
    plt.plot([min_x, max_x], [slope*min_x+intercept, slope*max_x+intercept], '-', color='k')
    plt.savefig(path_outputs + str(property_)+'_matern_kernel_CV_fig1.pdf', bbox_inches='tight', transparent=True)
    plt.show()
    return measured, predicted

In [11]:
# Prep y data for CV function
log_data = np.log(y)

In [8]:
# Perform ML model training
final_prams = ML_train(X, y)

Full GP regression model
Hyperparameters: 2.24125819484e-11 203.879134241


In [14]:
# Perform CV train
cross_validation(X=X, log_data=log_data, property_='activations')

ValueError: array must not contain infs or NaNs

In [16]:
np.isinf(log_data).any()

True